ISE 3230 Project Code

In [15]:
import cvxpy as cp
import numpy as np



# Distance matrix for the garage and 9 houses
distance_matrix = np.array([
    [0, 0.44, 3.43, 2.24, 2.54, 0.80, 0.66, 1.64, 0.28, 3.09],  # Garage row
    [0.44, 0, 3.41, 2.56, 2.18, 0.78, 0.29, 1.98, 0.17, 2.73],  # House 1 row
    [3.43, 3.41, 0, 2.67, 2.76, 2.66, 3.63, 2.76, 3.39, 2.75],  # House 2 row
    [2.24, 2.56, 2.67, 0, 3.76, 2.01, 2.85, 0.60, 2.43, 4.13],  # House 3 row
    [2.54, 2.18, 2.76, 3.76, 0, 2.00, 2.18, 3.36, 2.30, 0.56],  # House 4 row
    [0.80, 0.78, 2.66, 2.01, 2.00, 0, 1.04, 1.48, 0.74, 2.49],  # House 5 row
    [0.66, 0.29, 3.63, 2.85, 2.18, 1.04, 0, 2.25, 0.42, 2.74],  # House 6 row
    [1.64, 1.98, 2.76, 0.60, 3.36, 1.48, 2.25, 0, 1.83, 3.78],  # House 7 row
    [0.28, 0.17, 3.39, 2.43, 2.30, 0.74, 0.42, 1.83, 0, 2.85],  # House 8 row
    [3.09, 2.73, 2.75, 4.13, 0.56, 2.49, 2.74, 3.78, 2.85, 0]   # House 9 row
])

# Mowing time (in HOURS) for each house i
t_i = np.array([1/2, 1/2, 1/3, 3/4, 1/3, 1/3, 1/2, 1/3, 3/4]) # mowing times

# Blade switching times (in HOURS) for Blade A, Blade B, and Blade C
blade_change_times = np.array([1/4, 1/2, 3/4])  # Blade A: 1/4 hr, Blade B: 1/2 hr, Blade C: 3/4 hr

# Corresponding blade required for each house i (0: Blade A, 1: Blade B, 2: Blade C) and garage (-1)
blade_type = np.array([-1, 0, 1, 0, 2, 1, 0, 2, 1, 1]) 

# Cost factors for travel, mowing, and blade switching
travel_cost_factor = 0.81 # $/mile for total travel cost per mile (SUMS UP GAS AND LABOR COSTS PER MILE)
labor_cost_factor = 40 # labor cost per hour for 2 laborers
blade_change_costs = blade_change_times * labor_cost_factor  # Convert blade change times to costs



# Decision variables
n = len(distance_matrix)  # Number of houses (including garage)
x = cp.Variable((n, n), boolean=True) # binary decision variable for the decision of whether to travel from house i to house j
y = cp.Variable(n, integer=True) # integer variable for the subtour elimination constraints

# Costs
travel_cost = cp.sum(cp.multiply(distance_matrix, x)) * travel_cost_factor # total travel cost
mowing_cost = cp.sum([t_i[i-1] * cp.sum(x[i, :]) for i in range(1, n)]) * labor_cost_factor # total mowing cost

# Initial blade switching cost when going from garage to some house i
initial_blade_change_cost = 0
for j in range(1, n):  # Only considers paths from the garage to any house
    if blade_type[j] != -1:
        initial_blade_change_cost += x[0, j] * blade_change_costs[blade_type[j]]

# total blade switching cost (initial blade change cost + blade change cost for each house i)
blade_cost = initial_blade_change_cost
for i in range(1, n):  # From any house
    for j in range(1, n):  # To any house
        if i != j:
            blade_cost += x[i, j] * blade_change_costs[blade_type[j]] * (blade_type[i] != blade_type[j])

# Objective function -> The total cost of travel, mowing, and blade switching
obj_func = travel_cost + mowing_cost + blade_cost



# Constraints
constraints = []

# Each row must sum to 1 (one outgoing from each house i and garage)
for i in range(n):
    constraints.append(cp.sum(x[i, :]) == 1)
   
# Each column must sum to 1 (one incoming edge to each house i and garage)
for j in range(n):
    constraints.append(cp.sum(x[:, j]) == 1)

# No self-loops
for i in range(n):
    constraints.append(x[i, i] == 0) # Garage cannot travel to itself

# Subtour elimination using flow variables
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            constraints.append(y[i] - y[j] + n * x[i, j] <= n - 1)

# Ensures garage is the starting point
constraints.append(y[0] == 0)

# Flow variable bounds (to ensure route is consistent and not missing any houses)
for i in range(1, n):
    constraints.append(y[i] >= 1)
    constraints.append(y[i] <= n - 1)



# Solve the problem
problem = cp.Problem(cp.Minimize(obj_func), constraints) # minimize the objective function (total cost)
problem.solve(solver=cp.GUROBI, verbose=True)

# Results
if problem.status == cp.OPTIMAL:
   
    # Prints out each path in the optimal route that optimizes the total cost
    path = {i: j for i in range(n) for j in range(n) if x.value[i, j] > 0.5}
    optimal_path = [0]
    while len(optimal_path) < n + 1:
        optimal_path.append(path[optimal_path[-1]])
   
    print("\nPath Costs:")
    total_travel_cost = 0
    total_mowing_cost = 0
    total_blade_cost = 0
    
    for i in range(len(optimal_path) - 1):
        start = optimal_path[i]
        end = optimal_path[i + 1]
        
        travel = distance_matrix[start, end] * travel_cost_factor
        mowing = t_i[end - 1] * labor_cost_factor if end != 0 else 0
        blade = 0
        if start == 0:  # First path (garage to first house)
            blade = blade_change_costs[blade_type[end]]
        elif end != 0 and blade_type[start] != blade_type[end]:  # Skip last path to garage
            blade = blade_change_costs[blade_type[end]]

        total_travel_cost += travel
        total_mowing_cost += mowing
        total_blade_cost += blade
        
        print(f"\nPath {i + 1}: {start} -> {end}")
        print("Travel cost: ")
        print(round(travel, 2))
        print("Mowing cost: ")
        print(round(mowing, 2))
        print("Blade change cost: ")
        print(round(blade, 2))


    # Summary of Costs
    total_cost = total_travel_cost + total_mowing_cost + total_blade_cost
    
    print("\nSummary of Costs: ")
    print("Total travel cost: ")
    print(round(total_travel_cost, 2))
    print("Total mowing cost: ")
    print(round(total_mowing_cost, 2))
    print("Total blade maintenance cost: ")
    print(round(total_blade_cost, 2))
    print("Solver optimized total cost: ")
    print(round(problem.value, 2))
    print("Our optimized total cost: ")
    print(round(total_cost, 2))
       
    # Order the path
    path = []
    for i in range(n):
        for j in range(n):
            if x.value[i, j] > 0.5:
                path.append((i, j))
   
    # Reformat the ordered path
    ordered_path = [0]  # Start at the garage
    while len(ordered_path) < n + 1:  # Ensure all nodes are included and the route ends at the garage
        for i, j in path:
            if i == ordered_path[-1]:
                ordered_path.append(j)
                break
    print("\nOrdered path:")
    print(ordered_path)
   
else:
    print("No optimal solution found.")


                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Nov 25 06:31:09 PM: Your problem has 110 variables, 121 constraints, and 0 parameters.
(CVXPY) Nov 25 06:31:09 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 25 06:31:09 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 25 06:31:09 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 25 06:31:09 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 25 06:31:09 PM: Compiling problem (target solver=GUROBI).